# Neural Network

http://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py

신경망은 torch.nn패키지를 사용해 구축가능하다.

autograd를 체험해봤다면 모델을 구현하기위해 autograd를 이용해서 변수들을 미분한다. nn.Module은 레이어를 포함하고 forward(input)은 output을 리턴한다.

신경망의 전형적인 학습 과정은 다음과 같다.

* 학습가능한 파라미터를 가진 신경망을 정의한다.
* 데이터셋을 반복적으로 입력한다.
* 신경망을 통해 입력을 계산한다.
* Loss를 계산한다(얼마나 output이 정확한지)
* 신경망 파라미터를 backpropagation한다.
* 신경망의 가중치를 업데이트한다. 일반적으로는 weight = weight - learning_rate * gradient 로 업데이트한다.

![](http://pytorch.org/tutorials/_images/mnist.png)

### 신경망을 정의한다.

In [2]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 첫번째 입력은 1채널, 6 채널의 출력, 5x5 convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation : y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number.
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
net = Net()

In [5]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # Conv1's weight

10
torch.Size([6, 1, 5, 5])


In [6]:
input_ = Variable(torch.randn(1,1, 32, 32))
out = net(input_)
print(out)

Variable containing:
-0.0123 -0.0045 -0.0332 -0.0665  0.1261  0.0945  0.0469 -0.0184  0.0166 -0.0086
[torch.FloatTensor of size 1x10]



모든 파라미터의 버퍼를 0으로 초기화

In [7]:
net.zero_grad()
out.backward(torch.randn(1, 10))

torch.nn은 오직 미니-배치 를 지원한다. 전체 torch.nn패키지는 샘플의 미니배치만을 입력으로 지원한다. 싱글 샘플말고.

예를들어, nn.Conv2d는 4D 텐서를 입력받는다.
nSamples * nChannels * Height(row) * Width(col).

만약 싱글 샘풀을 갖고있다면, input.unsqueeze(0)을 써서 배치차원을 추가해주자.

다음으로 넘어가기전에 보충을 한번 하자.

* torch.Tensor - 다차원 array
* autograd.Variable - 텐서를 감싸고 적용된 연산의 순서를 기억한다. Tensor와 같은 API를 갖고 backward()가 있다. 
* nn.Module - 신경망 모듈. GPU로 옮기는 헬퍼함수, 외부저장, 로딩 등.. 파라미터를 캡슐화하는 편한 방식이다. 
* nn.Parameter - Variable 처럼 Module의 attribute로 등록된 파라미터들
* autograd.Function - autograd 연산의 전방/역방 정의의 구현.
최소 싱글 Function노드를 만드는 모든 Variable객체의 연산은 Variable이 만든 함수로 연결되어있고 그 순서를 암호화한다.

지금까지 우린 다음과 같은 사항을 진행해왔다.
* 신경망을 구현
* 입력을 처리하고 역전파 함수를 호출

아직 남은 부분은..
* loss 계산
* 신경망 가중치 업데이트

### Loss Function

Loss function은 입력을 넣은 출력값과 타겟값(실제값) 쌍을 받고 얼마나 출력값이 타겟값과 차이가 있는지 계산한다.

torch.nn 패키지에는 매우 다양한 loss function이 있다. 
간단한 Loss function은 nn.MSELoss 다. 출력과 타겟 간의 
mean-squared error를 계산한다. 

$$MSE = \frac{1}{n}\sum_n (Y_i - \hat{Y_i})^2$$

In [35]:
output = net(input_)
target = Variable(torch.arange(1, 11)) # 예를 들기위한 더미 타겟
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.2924
[torch.FloatTensor of size 1]



In [36]:
torch.sum((output - target)**2) / output.size()[1]

Variable containing:
 38.2924
[torch.FloatTensor of size 1]

만약 당신이 .grad_fn attribute를 사용해 loss를 역방향으로 보내고 싶다면, 연산 그래프를 볼것이다.

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d -> view -> linear -> relu -> linear -> relu -> linear -> MSELoss -> loss

loss.backward()를 호출할때 모든 그래프가 loss에 관해 미분된다.

그리고 그래프의 모든 Variable객체는 .grad 변수에 gradient를 통합한다.



In [37]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLu


### Backprop

에러를 역전파시키기위해 해야할 것은 loss.backward()이다. 

존재하는 gradient들을 비워야 통합된다.

In [38]:
net.zero_grad() # 모든 파라미터의 gradient버퍼를 0 초기화

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
 0.0563
-0.0446
-0.0238
-0.1304
 0.1107
-0.0657
[torch.FloatTensor of size 6]



In [39]:
net.conv2.bias.grad

Variable containing:
-0.0023
 0.0000
 0.0228
-0.0221
-0.0449
-0.0343
 0.0092
-0.0575
 0.2055
-0.0704
-0.1091
-0.0243
-0.0767
 0.1045
-0.0122
-0.1219
[torch.FloatTensor of size 16]

이제 신경망의 가중치를 업데이트 해보자.

### Update the weights

가장 단순한 업데이트 규칙은 Stochastic Gradient Descent 이다.

$$W = weight, L = learning rate, G = gradient$$
$$Weight = W - L * G$$

In [41]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

지금 당신은 신경망을 구현해보았지만, SGD, Nesterov-SGD, Adam, RMSProp, 등 다른 업데이트 규칙을 사용하고싶다면 torch.optim 패키지를 사용해보시라.

In [42]:
import torch.optim as optim

In [43]:
# Create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 트레이닝 루프에 추가
optimizer.zero_grad()
output = net(input_)
loss = criterion(output, target)
loss.backward()
optimizer.step()